In [21]:
import json
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import os
from sklearn.cluster import DBSCAN
# from matplotlib.cm import cm
# from scipy.stats import mode
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta


In [22]:


def calculate_centroids(data, labels):
    unique_labels = np.unique(labels)
    centroids = []

    for label in unique_labels:
        cluster_points = data[labels == label]
        centroid = np.mean(cluster_points, axis=0)
        centroids.append((label, centroid))
    return centroids

def find_closest_centroid(previous_centroids, current_centroids, id_mapping, next_id, frame_counts, frame_threshold=20):
    new_centroids = []
    for curr_label, curr_centroid in current_centroids:
        min_distance = float('inf')
        closest_centroid = None
        closest_prev_label = None

        for prev_label, prev_centroid in previous_centroids:
            distance = np.linalg.norm(curr_centroid - prev_centroid)
            if distance < min_distance:
                min_distance = distance
                closest_centroid = curr_centroid
                closest_prev_label = prev_label

        if closest_prev_label is not None and min_distance < 0.5:  # Threshold distance
            # Use existing ID if close to a previous centroid, initializing if necessary
            centroid_id = id_mapping.get(closest_prev_label, next_id)
            if closest_prev_label not in id_mapping:
                id_mapping[closest_prev_label] = centroid_id
                frame_counts[centroid_id] = 0  # Initialize frame count if first occurrence
            frame_counts[centroid_id] += 1  # Increment frame count
        else:
            # Assign a new ID if no close match found
            centroid_id = next_id
            id_mapping[curr_label] = centroid_id
            frame_counts[centroid_id] = 1  # Initialize frame count
            next_id += 1

        # Store the ID in the mapping and in the new_centroids list
        new_centroids.append((centroid_id, curr_centroid))

    # Filter centroids based on frame threshold
    valid_centroids = [(cid, c) for cid, c in new_centroids if frame_counts[cid] >= frame_threshold]
    return valid_centroids, next_id


In [23]:
visulization = False
previous_centroids = []
id_mapping = {}
frame_counts = {}
next_id = 0
filePath = "./data/drone_data_2.txt"
visFile = filePath.split("/")[-1].split(".")[0]
data = []


In [24]:

with open(filePath, "r") as file:
    for line in file:
        data.append(json.loads(line)["answer"])
radarData = pd.DataFrame(data)

newDF = radarData
radarData['datenow'] = pd.to_datetime(radarData['datenow'], format='%m/%d/%Y')
radarData['timenow'] = pd.to_datetime(radarData['timenow'].str.replace('_', ':'), format='%H:%M:%S').dt.time
radarData['datetime'] = radarData.apply(lambda row: datetime.combine(row['datenow'], row['timenow']), axis=1)
radarData['datetime'] = radarData['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
radarData = radarData[["datetime","dopplerIdx","x_coord","y_coord","z_coord", "rp_y","noiserp_y","snrDB","noiseDB"]]

start_time = radarData["datetime"][0]#add milisecond
start_time_obj = datetime.strptime(start_time,'%Y-%d-%m %H:%M:%S')
frameID = 0
time_frames = []
print(start_time_obj)


2024-10-05 17:29:19


In [30]:
for index,row in radarData.iterrows():
    time_current = start_time_obj+timedelta(seconds=frameID*(100)/1000)
    datetime_obj = pd.to_datetime(time_current, format='%Y-%d-%m %H:%M:%S.%f')
    new_datetime_obj = datetime_obj + relativedelta(months=1)
    new_timestamp = new_datetime_obj.strftime('%Y-%m-%d %H:%M:%S.%f')
    # time_frames.append(new_timestamp.strftime('%Y-%d-%m %H:%M:%S.%f'))
    radarData.loc[index, 'datetime']  = new_timestamp 

    frameID +=1

In [26]:

print("Dropping NAN rows")
radarData = radarData.dropna()
for index, row in radarData.iterrows():
    if sum(radarData["dopplerIdx"][index]) == 0:
        radarData.drop(index=index)
# radarData.to_csv("radarData.csv",index=False)

Dropping NAN rows


In [31]:

teleData = pd.read_csv('./telemetry_data/2024-11-05_17_29_23_telemtry.csv')
teleData = teleData[["datetime","x_m","y_m","z_m","roll_rad_s","pitch_rad_s","yaw_rad_s"]]

radarData['datetime'] = pd.to_datetime(radarData['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
teleData['datetime'] = pd.to_datetime(teleData['datetime'])
print(radarData['datetime'][7],teleData['datetime'][7])


2024-11-05 17:34:52.700000 2024-11-05 17:29:26.525936


In [32]:

mergedTeleRadar = pd.merge_asof(radarData, teleData, on='datetime', direction='nearest')
mergedTeleRadar =mergedTeleRadar.dropna(subset=['x_m'])
print("teleData.shape: ", teleData.shape)
print("radarData.shape: ", radarData.shape)
print("200ms - mergedTeleRadar after dropna.shape: ",mergedTeleRadar.shape)
# mergedTeleRadar.to_csv("mergedTeleRadar.csv",index=False)
# cmap = plt.colormaps.get_cmap("tab20", 20)
cmap = cm.get_cmap("tab20", 20)


teleData.shape:  (10306, 7)
radarData.shape:  (2336, 9)
200ms - mergedTeleRadar after dropna.shape:  (2336, 15)


/tmp/ipykernel_2393937/2826881113.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("tab20", 20)


In [33]:
mergedTeleRadar.head(5)

,datetime,dopplerIdx,x_coord,y_coord,z_coord,rp_y,noiserp_y,snrDB,noiseDB,x_m,y_m,z_m,roll_rad_s,pitch_rad_s,yaw_rad_s
0,2024-11-05 17:34:52.600,[0],[-0.8727030754089355],[0.08945613354444504],[-0.917457640171051],"[86.93939246479692, 92.27013197134659, 95.9075...","[35.35664841612515, 37.77115983968, 39.0881660...",[11.700000000000001],[59.900000000000006],0.081293,-1.934721,0.904508,0.013519,-0.029905,0.026537
1,2024-11-05 17:34:52.700,[0],[-0.8330347537994385],[0.17262676358222961],[-0.9421224594116211],"[86.90803517358194, 92.3014892625616, 95.90757...","[37.175371306595046, 34.4786442621052, 33.7887...",[12],[59.2],0.082730,-1.896414,0.905957,-0.041808,0.000712,0.002439
2,2024-11-05 17:34:52.800,[0],[-0.8330347537994385],[0.2129267454147339],[-0.9338398575782776],"[86.87667788236692, 92.27013197134659, 95.9075...","[36.45415360865008, 35.921079657995115, 38.272...",[11.3],[60],0.075372,-1.850321,0.908989,0.007556,0.018423,-0.011260
3,2024-11-05 17:34:52.900,[0],[-0.8330347537994385],[0.21448945999145508],[-0.9334821701049805],"[86.97074975601191, 92.20741738891661, 95.9075...","[35.04307550397517, 38.05337546061498, 38.7745...",[11.200000000000001],[59.900000000000006],0.067528,-1.813315,0.913229,0.042892,-0.004034,-0.041149
4,2024-11-05 17:34:53.000,[0],[-0.8727030754089355],[0.08166239410638809],[-0.9181841611862183],"[86.90803517358194, 92.20741738891661, 95.9075...","[35.07443279519017, 37.144014015380044, 39.433...",[11.600000000000001],[59.800000000000004],0.053172,-1.768407,0.923403,-0.047960,-0.011644,-0.000249


In [ ]:
if True : 
    for index,row in mergedTeleRadar.iterrows(): 
        frames = 10
        if index >= len(mergedTeleRadar) - frames:
            break
        x_coords = [e for elm in mergedTeleRadar['x_coord'][index:index + frames] for e in elm]
        y_coords = [e for elm in mergedTeleRadar['y_coord'][index:index + frames] for e in elm]
        z_coords = [e for elm in mergedTeleRadar['z_coord'][index:index + frames] for e in elm]
        doppler_idx = [e for elm in mergedTeleRadar['dopplerIdx'][index:index + frames] for e in elm]
        colors = [cmap(val % 20) for val in doppler_idx]
        sns.set(style="whitegrid")
        fig = plt.figure(figsize=(12,6))
        ax1 = fig.add_subplot(121,projection='3d')
        img1 = ax1.scatter(x_coords, y_coords, z_coords, c = colors, marker='o')
        fig.colorbar(img1)
        ax1.set_title('Radar PCD')
        ax1.set_xlabel('X')
        ax1.set_ylabel('Y')
        ax1.set_zlabel('Z')
        ax1.set_xlim(-10, 10)
        ax1.set_ylim(0, 10)
        ax1.set_zlim(-3, 10)

        data = np.array([x_coords, y_coords, z_coords]).T
        clustering = DBSCAN(eps=1, min_samples=15).fit(data)
        cluster_labels = np.array(clustering.labels_)
        current_centroids = calculate_centroids(data, cluster_labels)

        # Get the closest centroids and assign consistent IDs, keeping only valid ones
        valid_centroids, next_id = find_closest_centroid(previous_centroids, current_centroids, id_mapping, next_id, frame_counts, frame_threshold=10)
        previous_centroids = current_centroids  # Update previous centroids for next frame
        for centroid_id, centroid in valid_centroids:
            color = cmap(centroid_id % 20)  # Assign unique color based on ID
            ax1.scatter(centroid[0], centroid[1], centroid[2], marker='o', s=200, color=color, label=f'Cluster ID {centroid_id}')
        print(f"Index: {index}")
        ax2 = fig.add_subplot(122,projection='3d')
        x_t = [elm for elm in mergedTeleRadar['x_m'][index:index + frames]]
        print(x_t)
        y_t = [elm for elm in mergedTeleRadar['y_m'][index:index + frames]]
        z_t = [elm for elm in mergedTeleRadar['z_m'][index:index + frames]]  
        img2 = ax2.scatter(x_t, y_t, z_t,marker='o')
        fig.colorbar(img2)
        ax2.set_title('Telemetry UAV')
        ax2.set_xlabel('X')
        ax2.set_ylabel('Y')
        ax2.set_zlabel('Z')
        ax2.set_xlim(-10, 10)
        ax2.set_ylim(0, 10)
        ax2.set_zlim(-3, 10)


        plt.legend()
        plt.tight_layout()
        plt.savefig(f"./visualization/merged/radarDepth__{str(index)}.png")
        # plt.show()
        plt.close()
        # if index ==3:
        #     break
    print("Sample Visulization Saved")



No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Index: 0
[0.0812933444976806, 0.0827300772070884, 0.0753716751933097, 0.0675283744931221, 0.0531719624996185, 0.0358906500041484, 0.0138838831335306, -0.0107107441872358, -0.0394795536994934, -0.0704944059252739]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Index: 1
[0.0827300772070884, 0.0753716751933097, 0.0675283744931221, 0.0531719624996185, 0.0358906500041484, 0.0138838831335306, -0.0107107441872358, -0.0394795536994934, -0.0704944059252739, -0.1052102893590927]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


...

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Index: 92
[-0.907536804676056, -0.9068636298179626, -0.906182825565338, -0.9055215716362, -0.9049275517463684, -0.9043489098548888, -0.9038125276565552, -0.9032767415046692, -0.902776062488556, -0.9023273587226868]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Index: 93
[-0.9068636298179626, -0.906182825565338, -0.9055215716362, -0.9049275517463684, -0.9043489098548888, -0.9038125276565552, -0.9032767415046692, -0.902776062488556, -0.9023273587226868, -0.9018821120262146]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
